# Predicting Flight Delay

Problem Set-up:
We define a delayed flight to be one that is delayed by >= 15 minutes. 
The prediction problem is to train a model that can classify flights, to predict if they will or will not be delayed.

Use case:
- The idea is that this model would be useful to choosing airlines, flightpaths, airports, at the time of booking, relatively in advance of the scheduled departure (days, weeks, months ahead of time). Therefore, the prediction problem will focus on features that can be known in advance, rather than predicting using day-off features like weather and previous flights from that day. 

Notes:
- We restrict the analysis to relatively large airport, those with more than 20 (domestic) flights a day

# This notebook: more features
* one-hot day of week
* one-hot month
* one-hot airline
* one-hot airport
* one-hot departure hour

# Create separate models to predict for each airport 

Motivation:
The weights that should eb put on features (e.g. airlines) may differ depending on the airport (see e.g. NB 4B) since different airports can have different environments (e.g. San Diego and Chicago winters are very different; American Airlines is better in Tuscon than in Anchorage.

In [1]:
# Imports
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
sns.set_style('white')

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix

In [2]:
# Import custom code
from flightdelay.fld import io as flio

# Load data

In [3]:
airlines_df, airports_df, flights_df = flio.load_data()

In [4]:
# Filter data to keys of interest
keys = ['MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT',
       'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME',
       'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME',
       'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN',
       'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY']
flights_df = flights_df[keys]

# Remove airports with less than a certain number of flights

In [5]:
all_airports, airport_inverse, airport_count = np.unique(flights_df['ORIGIN_AIRPORT'],return_counts=True,return_inverse=True)

In [6]:
# Determine number of flights for the origin airport
Nflights_orig = np.zeros(len(airport_inverse))
for i in range(len(all_airports)):
    Nflights_orig[np.where(airport_inverse==i)] = airport_count[i]

In [7]:
flights_df = flights_df.loc[flights_df.index[Nflights_orig>=7300]]

# Remove cancelled flights

In [8]:
flights_df = flights_df.dropna()
flights_df.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,...,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY
0,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,...,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0
1,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,...,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0
2,1,1,4,US,840,N171US,SFO,CLT,20,18.0,...,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0
3,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,...,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0
4,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,...,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0


# Make train and val set

In [9]:
N_flights = 1000000
N_train = int(N_flights*.7)
np.random.seed(0)
flight_take_idx = np.random.permutation(flights_df.index)[:N_flights]
df_trainval = flights_df.loc[flight_take_idx]

In [10]:
df_trainval.head(20)

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,...,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY
3783504,8,23,7,WN,285,N7738A,MCI,DEN,1410,1408.0,...,1414.0,105.0,95.0,83.0,533,1437.0,6.0,1455,1443.0,-12.0
5412589,12,5,6,AS,335,N520AS,SJC,SEA,1550,1534.0,...,1554.0,120.0,135.0,105.0,697,1739.0,10.0,1750,1749.0,-1.0
5347613,12,1,2,UA,1750,N76254,SAN,IAH,1314,1354.0,...,1406.0,188.0,168.0,148.0,1303,1834.0,8.0,1822,1842.0,20.0
2383329,5,31,7,WN,396,N8318F,AUS,DEN,1935,2039.0,...,2051.0,140.0,128.0,109.0,775,2140.0,7.0,2055,2147.0,52.0
2817074,6,26,5,EV,4899,N133EV,ATL,MYR,1454,1509.0,...,1533.0,85.0,77.0,49.0,317,1622.0,4.0,1619,1626.0,7.0
5428910,12,6,7,OO,5236,N756SK,ORD,OKC,1913,1925.0,...,1949.0,133.0,121.0,91.0,693,2120.0,6.0,2126,2126.0,0.0
4073892,9,10,4,HA,516,N484HA,HNL,OGG,2150,2144.0,...,2155.0,38.0,41.0,23.0,100,2218.0,7.0,2228,2225.0,-3.0
2296450,5,26,2,EV,6176,N12996,EWR,GSO,1306,1318.0,...,1330.0,99.0,85.0,68.0,445,1438.0,5.0,1445,1443.0,-2.0
1182922,3,18,3,WN,1969,N932WN,TPA,MDW,1330,1330.0,...,1344.0,160.0,159.0,141.0,997,1505.0,4.0,1510,1509.0,-1.0
5228455,11,23,1,WN,2375,N8323C,IND,TPA,1545,1546.0,...,1555.0,135.0,119.0,105.0,837,1740.0,5.0,1800,1745.0,-15.0


# Randomly take subset of data

# Compute features
* one-hot day of week
* one-hot month
* one-hot airline
* one-hot airport
* one-hot departure hour

### 0. Original feat

In [11]:
original_feat = {'f_distance':df_trainval['DISTANCE'].values}

In [12]:
y_delay = df_trainval['DEPARTURE_DELAY'].values
y_delay01 = y_delay > 15

### 1. Day of week

In [13]:
def make_onehot_feat_dict(df, feat_key, feat_name):
    # Create features for each day of the week
    feat_vals = df[feat_key].values
    all_vals = np.unique(feat_vals)
    N_vals = len(all_vals)
    N_feat = N_vals - 1

    # Create keys
    keys = [0]*N_feat
    for i in range(N_feat):
        keys[i] = 'f_'+feat_name+'_'+ str(all_vals[i])

    # Create value for each training example in dict
    feat_dict = {}
    for i, k in enumerate(keys):
        this_day = all_vals[i]
        feat_dict[k] = feat_vals == this_day
    return feat_dict

In [14]:
daysfeat_dict = make_onehot_feat_dict(df_trainval, 'DAY_OF_WEEK', 'day')

### 2. Month

In [15]:
monthsfeat_dict = make_onehot_feat_dict(df_trainval, 'MONTH', 'month') 

### 3. Departing airport

In [16]:
dapfeat_dict = make_onehot_feat_dict(df_trainval, 'ORIGIN_AIRPORT', 'dap') 

### 4. Airline

In [17]:
alfeat_dict = make_onehot_feat_dict(df_trainval, 'AIRLINE', 'al') 

### 5. Departure hour

In [18]:
# Add departure hour as a feature
df_trainval['HOUR_DEPARTURE'] = np.floor(df_trainval['SCHEDULED_DEPARTURE'].values/100).astype(int)
df_trainval['HOUR_ARRIVAL'] = np.floor(df_trainval['SCHEDULED_ARRIVAL'].values/100).astype(int)
hrfeat_dict = make_onehot_feat_dict(df_trainval, 'HOUR_DEPARTURE', 'hr')

# Save new feature matrix

In [19]:
all_dicts = [original_feat, daysfeat_dict, monthsfeat_dict, dapfeat_dict, alfeat_dict, hrfeat_dict]
feat_dict = all_dicts[0].copy()
for d in all_dicts[1:]:
    feat_dict.update(d)

In [20]:
df_feat = pd.DataFrame.from_dict(feat_dict)
df_feat.head()

,f_al_AA,f_al_AS,f_al_B6,f_al_DL,f_al_EV,f_al_F9,f_al_HA,f_al_MQ,f_al_NK,f_al_OO,...,f_month_10,f_month_11,f_month_2,f_month_3,f_month_4,f_month_5,f_month_6,f_month_7,f_month_8,f_month_9
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


# Split data by airport

In [21]:
# zscore all features
from scipy.stats import zscore
df_feat_zscore = df_feat.apply(zscore)

In [22]:
airport_keys = np.array(df_feat.keys()[df_feat.columns.str.startswith('f_dap')])
feat_keys = np.array(df_feat.keys()[~df_feat.columns.str.startswith('f_dap')])
ap_keys = [s[-3:] for s in airport_keys]

N_airports = len(airport_keys)
airport_dfs = {}
airport_y = {}
for k in airport_keys:
    airport_dfs[k[-3:]] = df_feat_zscore[feat_keys][df_feat[k].values]
    airport_y[k[-3:]] = y_delay01[df_feat[k].values]
    
Nflights_by_airport = {}
for k in ap_keys:
    Nflights_by_airport[k] = len(airport_y[k])

# Train model for each airport

In [23]:
# Make train and validation sets
all_X_train = {}
all_X_val = {}
all_y_train = {}
all_y_val = {}
for i, ap in enumerate(ap_keys):
    # Compute number of training examples
    N_train = int(len(airport_dfs[ap])*.7)
    
    # Declare DFs
    all_X_train[ap] = airport_dfs[ap].loc[airport_dfs[ap].index.values[:N_train]].values
    all_X_val[ap] = airport_dfs[ap].loc[airport_dfs[ap].index.values[N_train:]].values

    # Determine labels
    all_y_train[ap] = airport_y[ap][:N_train]
    all_y_val[ap] = airport_y[ap][N_train:]

ABQ
ALB
ANC
ATL
AUS
BDL
BHM
BNA
BOI
BOS
BTR
BUF
BUR
BWI
CHS
CID
CLE
CLT
CMH
COS
CVG
DAL
DAY
DCA
DEN
DFW
DSM
DTW
ELP
EWR
FAT
FLL
GEG
GRR
GSO
HNL
HOU
HPN
IAD
IAH
ICT
IND
JAN
JAX
JFK
KOA
LAS
LAX
LGA
LGB
LIH
LIT
MAF
MCI
MCO
MDW
MEM
MIA
MKE
MSN
MSP
MSY
OAK
OGG
OKC
OMA
ONT
ORD
ORF
PBI
PDX
PHL
PHX
PIT
PSP
PVD
RDU
RIC
RNO
ROC
RSW
SAN
SAT
SAV
SDF
SEA
SFO
SJC
SJU
SLC
SMF
SNA
STL
TPA
TUL
TUS
TYS


In [25]:
all_models = {}
Cs = [.01, .1, 1, 10, 100, 1000, 10000, 100000]

for ap in ap_keys:
    print(ap)
    all_models[ap] = [0]*len(Cs)
    for i in range(len(Cs)):
        all_models[ap][i] = LogisticRegression(C=Cs[i])
        #model = LogisticRegression(C=C,fit_intercept=False)
        #model = LogisticRegression(C=C,class_weight='balanced')
        all_models[ap][i].fit(all_X_train[ap], all_y_train[ap])

ABQ
ALB
ANC
ATL
AUS
BDL
BHM
BNA
BOI
BOS
BTR
BUF
BUR
BWI
CHS
CID
CLE
CLT
CMH
COS
CVG
DAL
DAY
DCA
DEN
DFW
DSM
DTW
ELP
EWR
FAT
FLL
GEG
GRR
GSO
HNL
HOU
HPN
IAD
IAH
ICT
IND
JAN
JAX
JFK
KOA
LAS
LAX
LGA
LGB
LIH
LIT
MAF
MCI
MCO
MDW
MEM
MIA
MKE
MSN
MSP
MSY
OAK
OGG
OKC
OMA
ONT
ORD
ORF
PBI
PDX
PHL
PHX
PIT
PSP
PVD
RDU
RIC
RNO
ROC
RSW
SAN
SAT
SAV
SDF
SEA
SFO
SJC
SJU
SLC
SMF
SNA
STL
TPA
TUL
TUS
TYS


# Evaluate model for each C

In [26]:
ytrain_true = [0]*len(Cs)
yval_true = [0]*len(Cs)
ytrain_pred = [0]*len(Cs)
yval_pred = [0]*len(Cs)
ytrain_predp = [0]*len(Cs)
yval_predp = [0]*len(Cs)

for c_idx in range(len(Cs)):
    ytrain_true[c_idx] = np.zeros(0)
    ytrain_pred[c_idx] = np.zeros(0)
    ytrain_predp[c_idx] = np.zeros(0)
    for ap in all_models.keys():
        ytrain_true[c_idx] = np.append(ytrain_true[c_idx],all_y_train[ap])
        ytrain_pred[c_idx] = np.append(ytrain_pred[c_idx], all_models[ap][c_idx].predict(all_X_train[ap]))
        ytrain_predp[c_idx] = np.append(ytrain_predp[c_idx], all_models[ap][c_idx].predict_proba(all_X_train[ap])[:,1])

    yval_true[c_idx] = np.zeros(0)
    yval_pred[c_idx] = np.zeros(0)
    yval_predp[c_idx] = np.zeros(0)
    for ap in all_models.keys():
        yval_true[c_idx] = np.append(yval_true[c_idx],all_y_val[ap])
        yval_pred[c_idx] = np.append(yval_pred[c_idx], all_models[ap][c_idx].predict(all_X_val[ap]))
        yval_predp[c_idx] = np.append(yval_predp[c_idx], all_models[ap][c_idx].predict_proba(all_X_val[ap])[:,1])
        

In [41]:
ytrain_true[0]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [27]:
for c_idx in range(len(Cs)):
    print('accuracy: ',1-sum(np.abs(ytrain_pred[c_idx]-ytrain_true[c_idx]))/len(ytrain_true[c_idx]))

accuracy:  0.820760348182
accuracy:  0.821002260232
accuracy:  0.821053791792
accuracy:  0.821076694708
accuracy:  0.821076694708
accuracy:  0.821076694708
accuracy:  0.821076694708
accuracy:  0.821076694708


In [47]:
all_confuseT = [0]*len(Cs)
all_confuseV = [0]*len(Cs)
for c_idx in range(len(Cs)):
    all_confuseT[c_idx] = confusion_matrix(ytrain_true[c_idx], ytrain_pred[c_idx])
    all_confuseV[c_idx] = confusion_matrix(yval_true[c_idx], yval_pred[c_idx])
    
    print(Cs[c_idx])
    print('Train True positive rate:', all_confuseT[c_idx][1,1]/sum(all_confuseT[c_idx][1]))
    print('Train True negative rate:', all_confuseT[c_idx][0,0]/sum(all_confuseT[c_idx][0]))
    print(all_confuseT[c_idx])
    print('ROC AUC:', roc_auc_score(ytrain_true[c_idx], ytrain_predp[c_idx]))
    
    print('Train True positive rate:', all_confuseV[c_idx][1,1]/sum(all_confuseV[c_idx][1]))
    print('Train True negative rate:', all_confuseV[c_idx][0,0]/sum(all_confuseV[c_idx][0]))
    print(all_confuseV[c_idx])
    print('ROC AUC:', roc_auc_score(yval_true[c_idx], yval_predp[c_idx]))

0.01
Train True positive rate: 0.0135469843823
Train True negative rate: 0.997635409868
[[571683   1355]
 [123862   1701]]
ROC AUC: 0.690045715744
Train True positive rate: 0.0128349139122
Train True negative rate: 0.997360896398
[[244512    647]
 [ 53608    697]]
ROC AUC: 0.679114465691
0.1
Train True positive rate: 0.017521085033
Train True negative rate: 0.997059531829
[[571353   1685]
 [123363   2200]]
ROC AUC: 0.696078891032
Train True positive rate: 0.0160390387625
Train True negative rate: 0.996577731187
[[244320    839]
 [ 53434    871]]
ROC AUC: 0.682867391466
1
Train True positive rate: 0.018444924062
Train True negative rate: 0.996919925031
[[571273   1765]
 [123247   2316]]
ROC AUC: 0.696509897694
Train True positive rate: 0.0168308627198
Train True negative rate: 0.996357465971
[[244266    893]
 [ 53391    914]]
ROC AUC: 0.682707350869
10
Train True positive rate: 0.0186758838193
Train True negative rate: 0.996897238927
[[571260   1778]
 [123218   2345]]
ROC AUC: 0.6965357